In [170]:
import sqlite3
import re

In [171]:
def regexp(regpat, item):
    mypattern = re.compile(regpat)
    return mypattern.match(item) is not None

In [172]:
def regreplace(pattern, rep, item):
    mypattern = re.compile(pattern)
    return mypattern.sub(rep, item)

In [173]:
sqliteConnection = sqlite3.connect('../backend-db_test.db')
sqliteConnection.create_function("REGEXP", 2, regexp)
sqliteConnection.create_function("REGREPLACE", 3, regreplace)
cursor = sqliteConnection.cursor()

In [174]:
results = cursor.execute("SELECT * FROM words WHERE wordform REGEXP ?;", [r'^(b)arn$'])

OperationalError: database is locked

In [98]:
for result in results:
    print(result)

(98757, 'barn', 'NN', 'SIN-PLU|IND|NOM|NEU', 'INFL', 'barn', 'NN', '', '', '', 'inflector_no', 'Neutral', 'INFLECTED|BASE', 's13b1-barn', '21|23', '', '')
(98758, 'barn', 'NN', '|||NEU', 'LEX', 'barn', 'NN', '', '', '', 'ref_dic', '', '', '', '', '', '')


In [104]:
cursor.execute("SELECT COUNT(*) FROM n_spoken;").fetchall()

[(792936,)]

In [175]:
def reset_table(table, connection):
    cursor = connection.cursor()
    whitelist = ["e_spoken", "e_written", "sw_spoken", "sw_written", 
                "w_spoken", "w_written", "t_spoken", "t_written", 
                "n_spoken", "n_written"]
    if not table in whitelist:
        raise ValueError(f"Resetting {table} is not permitted")
    else:
        cursor.execute(f"DELETE FROM {table};")
        cursor.execute(f"INSERT INTO {table} SELECT * FROM base;")
        connection.commit()

In [176]:
reset_table('w_spoken', sqliteConnection)

OperationalError: database is locked

In [157]:
cursor.execute("UPDATE w_written SET nofabet = REGREPLACE(?,?, nofabet);", (r'R N', r'\bRN\b'))

In [158]:
results2 = cursor.execute("SELECT w.word_id, w.wordform, p.nofabet FROM words w LEFT JOIN w_written p ON w.word_id = p.word_id WHERE wordform = ?;", ('barn',)).fetchall()

In [159]:
print(results2)

[(98757, 'barn', '\\1(\x08)\x08RN\x08(\x08)\\1'), (98758, 'barn', '\\1(\x08)\x08RN\x08(\x08)\\1')]


In [165]:
regreplace(r'\bRN\b', r'R N', 'B AA1 RN A')

'B AA1 R N A'

In [169]:
sqliteConnection.close()